# Using the OSF API

For more information, visit the full [OSF API docs](http://developer.osf.io)!


This tutorial will start with some basic API queries to get publicly available information about the content on the OSF, and then move on to creating a project and adding a file via API.

Results from the OSF API mostly follows the [JSON API Specification](http://jsonapi.org/).

## Basic API Actions: Get Some Information

First, let's practice querying the OSF API for some public projects.

Since querying for public projects does not require us to provide authentication credentials, we can get right to the data without a lot of overhead.

First, some setup.

In [1]:
import json

import requests


OSF_API_URL = 'https://api.osf.io/v2/'


# A helper function to print out our API responses
def pretty_print(json_data):
    print(json.dumps(json_data, indent=4))

### Accessing the Public Nodes List

The following code will request the API endpoint for a list of nodes, a term we use for the overarching project. 

In [2]:
# Get a list of the first 10 public projects (called nodes)
results = requests.get(OSF_API_URL + 'nodes/').json()['data']

# Print out the data from the first of those 10 projects
pretty_print(results[0])

{
    "relationships": {
        "files": {
            "links": {
                "related": {
                    "href": "https://api.osf.io/v2/nodes/cy4zg/files/", 
                    "meta": {}
                }
            }
        }, 
        "view_only_links": {
            "links": {
                "related": {
                    "href": "https://api.osf.io/v2/nodes/cy4zg/view_only_links/", 
                    "meta": {}
                }
            }
        }, 
        "identifiers": {
            "links": {
                "related": {
                    "href": "https://api.osf.io/v2/nodes/cy4zg/identifiers/", 
                    "meta": {}
                }
            }
        }, 
        "draft_registrations": {
            "links": {
                "related": {
                    "href": "https://api.osf.io/v2/nodes/cy4zg/draft_registrations/", 
                    "meta": {}
                }
            }
        }, 
        "contributors": {
            "

In [3]:
# Here is each section of a result

pretty_print(results[0].keys())

[
    "relationships", 
    "attributes", 
    "type", 
    "id", 
    "links"
]


In [4]:
# Attributes has a lot of basic info like title, description, date modified, and tags
for result in results:
    pretty_print(result['attributes']['title'])

"Multivariate statistics practical in R 2017/fall"
"Open Source Dual Axis Gimbal System"
"FluoPI: Low cost and open source multi-fluorescence imaging system for teaching and research in biology and bioengineering"
"test-oem"
"Empirical studies"
"Codes: False alarm? A comprehensive reanalysis of \u201cEvidence that psychopathology symptom networks have limited replicability\u201d by Forbes, Wright, Markon, and Krueger."
"OSF API Webinar Materials (Oct 2017)"
"Pembenihan Lobster Air Tawar"
"Replication of O'Brien (2015) for Stanford Psych 251"
"Open Source Implicit Relational Assessment Procedure"


In [5]:
# We can also filter this list of nodes to return only certain results

url = OSF_API_URL + 'nodes/?filter[tags]=climate'

results = requests.get(url).json()['data']

for result in results:
    print('Title: {}, \nTags: {} \n\n'.format(result['attributes']['title'], result['attributes']['tags']))

Title: Changes of Tropical Cyclone Size in three Oceanic Basins of the Northern Hemisphere since 2001, 
Tags: [u'Climate', u'Tropical cyclones'] 


Title: Tropical weathering of the Taconic orogeny as a driver for Ordovician cooling, 
Tags: [u'climate', u'geochemistry', u'Ordovician', u'paleomagnetism', u'tectonics'] 


Title: Decreasing cloud cover drives the recent mass loss on the Greenland Ice Sheet, 
Tags: [u'Climate', u'Clouds', u'Greenland', u'Melt', u'Meteorology', u'Sea level', u'Surface mass balance'] 


Title: J2: an evaluation of new estimates from GPS, GRACE and load models compared to SLR, 
Tags: [u'climate', u'Global Positioning System (GPS)', u'Gravity Recovery and Climate Experiment (GRACE)', u'J2', u'oblateness', u'Satellite Laser Ranging (SLR)', u'seasonal', u'surface mass loading'] 


Title: Global Increases in Individualism, 
Tags: [u'Change over Time', u'Climate', u'Cross-cultural Differences', u'Cross-cultural Psychology', u'Cultural Change', u'Disaster', u'Indiv

### Pagination and Requesting More Results

The basic query will return the most recent 10 results. To get more results, you can either use the links provided in the API response to paginate through the returned pages, or alter the URL query parameters to request more results. The maximum number of results you may request at a time is 100.

The result's "Links" section will give you links for the first, last, next, and previous pages. It will also contain information about the total number of results, and how many results were returned with this query.

In [6]:
# Use the links at the bottom of the page to paginate through the returned results

url = OSF_API_URL + 'nodes/?filter[title]=fish'

# Basic query for first page
results = requests.get(url).json()
pretty_print(results['links'])

print('---------------------------------------')

# Get the next page results from the JSON
next_page = results['links']['next']
second_page_results = requests.get(next_page).json()

pretty_print(second_page_results['links'])

{
    "next": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish&page=2", 
    "meta": {
        "per_page": 10, 
        "total": 50
    }, 
    "prev": null, 
    "last": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish&page=5", 
    "first": null
}
---------------------------------------
{
    "next": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish&page=3", 
    "meta": {
        "per_page": 10, 
        "total": 50
    }, 
    "prev": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish", 
    "last": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish&page=5", 
    "first": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=fish"
}


In [7]:
# Use the page size query parameter to ask for more total results

url = OSF_API_URL + 'nodes/?filter[title]=science&page[size]=30'
results = requests.get(url).json()

pretty_print(results['links'])

{
    "next": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=science&page=2&page%5Bsize%5D=30", 
    "meta": {
        "per_page": 30, 
        "total": 839
    }, 
    "prev": null, 
    "last": "https://api.osf.io/v2/nodes/?filter%5Btitle%5D=science&page=28&page%5Bsize%5D=30", 
    "first": null
}


### Relationships

Some of the information on each project is stored in the relationships section of the result.

A relationship is a link to another API endpoint with much more information about that particular subject. For example, the contributors endpoint links to more information about each user who is a contributor on that project.

In [8]:
first_result = results['data'][0]

pretty_print(first_result['relationships']['contributors'])

print('---------------------->')
contributors_data = requests.get(first_result['relationships']['contributors']['links']['related']['href'])


pretty_print(contributors_data.json()['data'][0]['embeds']['users']['data'])

{
    "links": {
        "related": {
            "href": "https://api.osf.io/v2/nodes/hnp6x/contributors/", 
            "meta": {}
        }
    }
}
---------------------->
{
    "relationships": {
        "quickfiles": {
            "links": {
                "download": {
                    "href": "https://files.osf.io/v1/resources/5yjfut672z/providers/osfstorage/?zip=", 
                    "meta": {}
                }, 
                "upload": {
                    "href": "https://files.osf.io/v1/resources/5yjfut672z/providers/osfstorage/", 
                    "meta": {}
                }, 
                "related": {
                    "href": "https://api.osf.io/v2/users/qthsf/quickfiles/", 
                    "meta": {}
                }
            }
        }, 
        "nodes": {
            "links": {
                "related": {
                    "href": "https://api.osf.io/v2/users/qthsf/nodes/", 
                    "meta": {}
                }
            }
 

## Create a Project, Upload a File

Before starting this tutorial, make sure to [create an account on the osf](https://osf.io), login to that account, and create an API token by [visitng your settings page](https://osf.io/settings/tokens/).

Save your token as an enviornment variable, or replace the enviornment variable below with the text version of your token for local testing.

Here's an example of how to create a project (called a node) on the OSF, and then follow the API relationships to upload a file.

This is a python implementation of a guide found on the OSF [detailing a typical OSF Workflow](https://osf.io/y9jdt/wiki/Typical%20Workflow/)

In [10]:
import os

OSF_TOKEN = os.environ['OSF_TOKEN']  # replace this line with your token as a string instead if you like

In [11]:
# Let's defne a few helper functions to make sending data and credentials easier
def post_request(url, data):
    headers = {
        'Content-Type': 'application/vnd.api+json',
        'Authorization': 'Bearer {}'.format(OSF_TOKEN)
    }
    data = json.dumps(data)
    return requests.post(url, headers=headers, data=data)

In [12]:
def get_request(url):
    headers = {'Authorization': 'Bearer {}'.format(OSF_TOKEN)}
    return requests.get(url, headers=headers)

In [13]:
def put_request(url, data):
    headers = {
        'Content-Type': 'application/vnd.api+json',
        'Authorization': 'Bearer {}'.format(OSF_TOKEN)
    }
    data = json.dumps(data)
    return requests.put(url, headers=headers, data=data)

In [14]:
# Define the data for the node we'd like to create
node_data = {
    "data": {
        "type":"nodes",
        "attributes": {
            "title":"Test Project",
            "description": "This is a node created as an example of how to create a node!",
            "public": False,
            "category":"project"
        }
    }
}

In [15]:
# Post the data, get a response back with details about our node
node_response = post_request(OSF_API_URL + 'nodes/', node_data)
pretty_print(node_response.json())


node_response = get_request(OSF_API_URL + 'nodes/')

{
    "data": {
        "relationships": {
            "files": {
                "links": {
                    "related": {
                        "href": "https://api.osf.io/v2/nodes/qcrjw/files/", 
                        "meta": {}
                    }
                }
            }, 
            "view_only_links": {
                "links": {
                    "related": {
                        "href": "https://api.osf.io/v2/nodes/qcrjw/view_only_links/", 
                        "meta": {}
                    }
                }
            }, 
            "identifiers": {
                "links": {
                    "related": {
                        "href": "https://api.osf.io/v2/nodes/qcrjw/identifiers/", 
                        "meta": {}
                    }
                }
            }, 
            "draft_registrations": {
                "links": {
                    "related": {
                        "href": "https://api.osf.io/v2/nodes/qcrjw/draft_re

In [16]:
# Find the files relationship, follow the related -> href link
files_link = node_response.json()['data'][0]['relationships']['files']['links']['related']['href']
files_response = get_request(files_link).json()
pretty_print(files_response)

{
    "data": [
        {
            "relationships": {
                "files": {
                    "links": {
                        "related": {
                            "href": "https://api.osf.io/v2/nodes/qcrjw/files/osfstorage/", 
                            "meta": {}
                        }
                    }
                }
            }, 
            "attributes": {
                "node": "qcrjw", 
                "path": "/", 
                "kind": "folder", 
                "name": "osfstorage", 
                "provider": "osfstorage"
            }, 
            "type": "files", 
            "id": "qcrjw:osfstorage", 
            "links": {
                "storage_addons": "https://api.osf.io/v2/addons/?filter%5Bcategories%5D=storage", 
                "upload": "https://files.osf.io/v1/resources/qcrjw/providers/osfstorage/", 
                "new_folder": "https://files.osf.io/v1/resources/qcrjw/providers/osfstorage/?kind=folder"
            }
        }

In [17]:
# Find the upload link for OSF Storage in that list - should be the first element in the list for new nodes
# A node can have several external storage providers configured
upload_link = files_response['data'][0]['links']['upload']
upload_link

u'https://files.osf.io/v1/resources/qcrjw/providers/osfstorage/'

In [18]:
# Upload the file along with the kind and file name

upload_link_with_filename = upload_link + '?kind=file&name=newest_file.txt'
file_data = 'This is the entirety of the contents of the file I am uploading. It could have been more, but for an example, a small file seems like a better idea.'

put_response = put_request(upload_link_with_filename, file_data)
pretty_print(put_response.json())

{
    "data": {
        "attributes": {
            "kind": "file", 
            "resource": "qcrjw", 
            "name": "newest_file.txt", 
            "extra": {
                "hashes": {
                    "sha256": "9a4012e8fcbb3353bda8e2520976d3fc050aa4e43b7c1eb8d2d3acc882efab8b", 
                    "md5": "5ae2d0f6cf6e1de68b81df1c25728297"
                }, 
                "guid": null, 
                "checkout": null, 
                "downloads": 0, 
                "version": 1
            }, 
            "materialized": "/newest_file.txt", 
            "created_utc": null, 
            "modified": "2017-12-08T08:18:33.236486+00:00", 
            "etag": "8f63dba0a846be64eef90691fd7f39691a25faf3e7c4d2a7e99fa00f04d21258", 
            "provider": "osfstorage", 
            "contentType": "application/octet-stream", 
            "path": "/5a2a4ad9d7a04c000e8cbec1", 
            "modified_utc": "2017-12-08T08:18:33+00:00", 
            "size": 150
        }, 
        "

## You did it!
Visit your project on the OSF and see your newly updated file!

In [19]:
# Check our your project on the OSF by visiting the project's link
node_response.json()['data'][0]['links']['html']

u'https://osf.io/qcrjw/'